 ## test10.ipynb

Testing the various versions of the double sigma units, and networks that use them.

By `Sergio Verduzco`  
May, June 2018

In [1]:
%cd ..
from draculab import *
import numpy as np

/home/z/projects/draculab


### Test 0
Check the `port_idx` lists created after connecting double sigma units.

In [2]:
# Create two groups of 3 double sigma units 
net_params = {'min_delay' : 0.02, 'min_buff_size' : 8 } # parameter dictionary for the network
unit_params = {'type' : unit_types.double_sigma, 
               'init_val' : 0.5, 
               'n_ports' : 3,
               'slope_out' : 1.,
               'thresh_out' : 0.,
               'tau' : 0.01,
               'branch_params' : {
                   'branch_w' : [0.2, 0.3, 0.5],
                   'slopes' : 1.,
                   'threshs' : 0.
                                  }
              }
net = network(net_params)  # creating a newtwork
units1 = net.create(3,unit_params) # first group
units2 = net.create(3,unit_params) # second group

# connect the units
conn_spec = { 'rule' : 'all_to_all',
              'delay' : net_params['min_delay'],
              'allow_autapses' : True }
syn_spec = { 'type' : synapse_types.static,
             'init_w' : 1.0,
             'inp_ports' : [0, 1, 2]*3 } # the 3 units will project to ports 0, 1, 2
net.connect(units1, units2, conn_spec, syn_spec) 

syn_spec['inp_ports'] = [0, 2, 0]*3 # first unit projects to port 0, second to port 2, third to port 0
net.connect(units2, units1, conn_spec, syn_spec) # second unit projects to port 1

syn_spec['inp_ports'] = 0
net.connect([units1[0]], units1, conn_spec, syn_spec)

# Predicting these is a mind twister. You should have: 
# net.units1[0].port_idx = [ [0, 1, 2, 3], [], [] ]   -- remember these are indexes, not unit IDs
# net.units1[1].port_idx = [ [3], [], [0, 1, 2] ]
# net.units1[2].port_idx = [ [0, 1, 2, 3], [], []]
# net.units2[0].port_idx = [ [0, 1, 2], [], [] ]
# net.units2[1].port_idx = [ [], [0, 1, 2], [] ]
# net.units2[2].port_idx = [ [], [], [0, 1, 2] ]

units1_port_idx = ( [ [0, 1, 2, 3], [], [] ], [ [3], [], [0, 1, 2] ], [ [0, 1, 2, 3], [], [] ] )
units2_port_idx = ( [ [0, 1, 2], [], [] ], [ [], [0, 1, 2], [] ], [ [], [], [0, 1, 2] ] )


In [3]:
# compare with predicted values
for i in [0,1,2]:
    print(units1_port_idx[i] == net.units[units1[i]].port_idx)
    print(units2_port_idx[i] == net.units[units2[i]].port_idx)


True
True
True
True
True
True


In [4]:
net.units[4].get_mp_inputs(0.)

[array([], dtype=float64), array([ 0.5,  0.5,  0.5]), array([], dtype=float64)]

In [5]:
net.units[4].get_mp_weights(0.)

[array([], dtype=float64), array([ 1.,  1.,  1.]), array([], dtype=float64)]

In [5]:
# Implement AND and OR units